In [1]:
import pandas as pd
import numpy as np
import os
from typing import Dict, List
from pandas import DataFrame, to_datetime, Series

from utils.db.mysql_utils import select_multiple_results, select_single_result
from utils.dbqueries import datasetByTypeQuery1

from sqlite3 import Connection
from utils.common_constants import  COMPANY_ID,  END_DATE,LAST_DATA_DATE, DATE, PARAM_NAME,  LP_GROUP_ID, VALUE, GROSS_MARGIN_PARAMETER_NAME

# from utils.loader.last_data_date import get_last_data_date
import logging
from utils.errors import MissingData
from utils.config import processed_data_path, MEDIAN_COMPETITOR_COUNT, CURRENT_COMPETITOR_COUNT
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from utils.connection import get_db_connection

import matplotlib.pyplot as plt
from scipy.stats import gamma

/home/gagan/Documents/data_science/projects/snp_data/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
company_id = 6135
end_date = '2023-06-01'
lp_group_id = 11869

PRIMARY_REV_REC_REV_TYPE = 'revenue_per_month_rev_rec'
SECONDARY_REV_REC_REV_TYPE = 'Revenue Per Month Rev Rec'

PRIMARY_TYPE = 'primary_type'
SECONDARY_TYPE = 'secondary_type'

# last_data_date = get_last_data_date(company_id)
# last_data_date


In [3]:
def _arrange_data_from_db(data: List[Dict[str, str]]) -> Series:
    data_df = DataFrame(data, columns=[
    "company_id", 
    "company_name",
    "year_founded",
    "ticker_symbol",
    "exchange_name",
    "exchange_symbol",
    "website",
    "industry_group",
    "description",
    "fiscal_year", 
    "fiscal_quarter", 
    "TEV_90D_AVG_REV",
    "Total Revenues, 1 Yr. Growth % (TTM)",
    "Gross Profit Margin % (TTM)",
    "ltm_revenue",
    "COUNTRY",
    "ebitda_margin",
    "market_cap",
    "share_price",
    "companytype",
    "companystatustype"
    
])
    return data_df.squeeze()

In [4]:
def _raise_missing_data(self, msg: str, extra_params: dict = None):
        params = {COMPANY_ID: self.company_id,
                  END_DATE: self.end_date,
                  LAST_DATA_DATE: self.last_data_date,
                  LP_GROUP_ID: self.lp_group_id}
        logging.error(msg=msg, extra=params if not extra_params else params.update(extra_params))
        raise MissingData(msg)


In [5]:
def _get_snp_public_data(conn: Connection,
                               company_id: int) -> Series:
        params = {"PERIODTYPEID": 4}
        snp_data = select_multiple_results(conn=conn, sql_query=datasetByTypeQuery1)
        # display(revrec_revenue)
        if not snp_data:
            msg = f'There is no snp data avilable at this time. ' \
                  f'{end_date.replace(day=1)}, which is necessary for equity return input.'
            _raise_missing_data(msg=msg)
        snp_data = _arrange_data_from_db(snp_data)
        return snp_data

In [6]:
with get_db_connection() as conn:
    snp_data = _get_snp_public_data(conn=conn,
                                         company_id=company_id)
    display(snp_data)

UndefinedColumn: column "cb_categories" does not exist
LINE 26:     CB_CATEGORIES
             ^


In [ ]:
# snp_data.to_csv('snp_public_companies_final.csv')

In [ ]:
snp_data[snp_data['ticker_symbol'].isin(['EPAY', 'PAYA', 'QUOT', 'TUBE', 'RDEN'])]

,company_id,company_name,year_founded,ticker_symbol,exchange_name,exchange_symbol,website,industry_group,description,fiscal_year,fiscal_quarter,TEV_90D_AVG_REV,"Total Revenues, 1 Yr. Growth % (TTM)",Gross Profit Margin % (TTM),ltm_revenue,COUNTRY,ebitda_margin,market_cap,share_price,companytype


In [ ]:
snp_data['exchange_name'].unique()

array(['Warsaw Stock Exchange', 'Euronext Paris', 'Nasdaq Capital Market',
       'Nasdaq Global Market', 'Nasdaq Global Select',
       'XETRA Trading Platform', 'Australian Securities Exchange',
       'Singapore Exchange', 'The Stock Exchange of Hong Kong Ltd.',
       'Bolsas y Mercados Espanoles', 'London Stock Exchange AIM Market',
       'National Stock Exchange of India',
       'OMX Nordic Exchange Stockholm', 'Mumbai Stock Exchange',
       'The Tokyo Stock Exchange', 'The Tel-Aviv Stock Exchange',
       'New York Stock Exchange', 'Bolsa de Valores de Sao Paulo',
       'London Stock Exchange', 'The Toronto Stock Exchange',
       'Euronext Amsterdam', 'OMX Nordic Exchange Copenhagen',
       'OMX Nordic Exchange Helsinki', 'SIX Swiss Exchange',
       'Euronext Brussels', 'Indonesia Stock Exchange'], dtype=object)

In [ ]:
snp_data[snp_data['companytype'].isin(['Public Company'])]

,company_id,company_name,year_founded,ticker_symbol,exchange_name,exchange_symbol,website,industry_group,description,fiscal_year,fiscal_quarter,TEV_90D_AVG_REV,"Total Revenues, 1 Yr. Growth % (TTM)",Gross Profit Margin % (TTM),ltm_revenue,COUNTRY,ebitda_margin,market_cap,share_price,companytype
0,43288520,01Cyberaton Proenergy S.A.,2007.0,TNT,Warsaw Stock Exchange,WSE,01c.eu,Independent Power and Renewable Electricity Pr...,01Cyberaton Proenergy S.A. engages in the impl...,2023,2,12.142352,-86.8314,198.8950,1.276341,Poland,-55.5701,16.127901,0.984034,Public Company
1,25617678,1000mercis,2000.0,ALMIL,Euronext Paris,ENXTPA,numberly.com/fr,Media,1000mercis provides marketing solutions for co...,2023,2,0.935712,5.3172,19.8596,85.503050,France,4.4349,65.103080,30.117853,Public Company
2,689627552,10X Capital Venture Acquisition Corp,2020.0,VCVC,Nasdaq Capital Market,NasdaqCM,www.10xspac.com,Capital Markets,"As of July 22, 2021, 10X Capital Venture Acqui...",2020,4,NaN,NaN,NaN,NaN,United States,NaN,NaN,NaN,Public Company
3,706460426,10X Capital Venture Acquisition Corp. II,2021.0,AAGR,Nasdaq Global Market,NasdaqGM,www.10xcapital.com,Capital Markets,10X Capital Venture Acquisition Corp. II does ...,2023,3,NaN,NaN,NaN,NaN,United States,NaN,71.711020,NaN,Public Company
4,223288117,"10x Genomics, Inc.",2012.0,TXG,Nasdaq Global Select,NasdaqGS,www.10xgenomics.com,Life Sciences Tools and Services,"10x Genomics, Inc., a life science technology ...",2023,3,8.968125,17.3261,69.8730,590.980000,United States,-26.5210,4843.501500,NaN,Public Company
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25556,717662404,"Zylox-Tonbridge Medical Technology Co., Ltd.",2012.0,2190,The Stock Exchange of Hong Kong Ltd.,SEHK,www.zyloxtb.com,Health Care Equipment and Supplies,"Zylox-Tonbridge Medical Technology Co., Ltd., ...",2023,2,3.170676,58.7669,74.8127,56.711000,China,-49.4962,527.602840,1.600302,Public Company
25557,43279463,Zymeworks Inc.,2003.0,ZYME,Nasdaq Global Select,NasdaqGS,www.zymeworks.com,Biotechnology,"Zymeworks Inc., a clinical-stage biopharmaceut...",2023,3,0.838585,1445.8622,63.0425,461.579000,United States,45.6526,430.021200,NaN,Public Company
25558,3103657,"Zynex, Inc.",1996.0,ZYXI,Nasdaq Global Select,NasdaqGS,www.zynex.com,Health Care Equipment and Supplies,"Zynex, Inc., together with its subsidiaries, d...",2023,3,1.889007,24.1197,80.0244,185.842000,United States,12.1049,280.800000,NaN,Public Company
25559,1370715,Zytronic plc,1999.0,ZYT,London Stock Exchange AIM Market,AIM,zytronic.co.uk,"Electronic Equipment, Instruments and Components","Zytronic plc, together with its subsidiaries, ...",2023,4,0.139643,-30.2270,24.5063,10.504355,United Kingdom,-7.7003,10.847814,NaN,Public Company


In [ ]:
snp_data['exchange_symbol'].unique()

array(['WSE', 'ENXTPA', 'NasdaqCM', 'NasdaqGM', 'NasdaqGS', 'XTRA', 'ASX',
       'SGX', 'SEHK', 'BME', 'AIM', 'NSEI', 'OM', 'BSE', 'TSE', 'TASE',
       'NYSE', 'BOVESPA', 'LSE', 'TSX', 'ENXTAM', 'CPSE', 'HLSE', 'SWX',
       'ENXTBR', 'IDX'], dtype=object)

In [ ]:
el = ['NYSE', 'NasdaqGS', 'NasdaqCM', 'NasdaqGM', 'TSX', 'LSE', 'AIM', 'SWX', 'XTRA', 'NSEI', 'BSE', 'TSE', 'ENXTPA', 'ENXTBR', 'ENXTAM', 'ASX', 'BME', 'BOVESPA', 'SEHK', 'SGX', 'CPSE', 'HLSE', 'OM', 'WSE', 'IDX', 'TASE']

for e in el:
    if(e in snp_data['exchange_symbol'].unique()):
        print(e, "is present in the list")

NYSE is present in the list
NasdaqGS is present in the list
NasdaqCM is present in the list
NasdaqGM is present in the list
TSX is present in the list
LSE is present in the list
AIM is present in the list
SWX is present in the list
XTRA is present in the list
NSEI is present in the list
BSE is present in the list
TSE is present in the list
ENXTPA is present in the list
ENXTBR is present in the list
ENXTAM is present in the list
ASX is present in the list
BME is present in the list
BOVESPA is present in the list
SEHK is present in the list
SGX is present in the list
CPSE is present in the list
HLSE is present in the list
OM is present in the list
WSE is present in the list
IDX is present in the list
TASE is present in the list
